# Check embeddings available for all years from 2000

In [2]:
# Reload modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

# Import from vm notebook dir
import sys; sys.path.insert(0, '../../')

import os
from datetime import datetime

from ExplainingDriftTextEmbeddings.access.file_storage import FileStorage
from ExplainingDriftTextEmbeddings.access.amazon_pickle_reader import AmazonPickleReader
from ExplainingDriftTextEmbeddings.access.interim_storage import InterimStorage

from BenchTest.amore.amazon_reviews_reader import AmazonReviewsReader

In [3]:
file_storage = FileStorage()

In [4]:
amazon_raw_id = 'amazon_raw.pickle'
print(file_storage.get_filepath(amazon_raw_id))

amazon_directoy = os.path.dirname(file_storage.get_filepath(amazon_raw_id))
print(amazon_directoy)

amazon_reader = AmazonPickleReader(amazon_directoy)

/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/clustering/amazon_raw.pickle
/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/clustering


## Get complete ID mapping

In [4]:
interim_storage = InterimStorage('originalno_to_rawid')
print(interim_storage.get_filepath())

if(not interim_storage.isfile()):

    # Config
    amazon_raw_id = 'amazon_raw.pickle'
    print(file_storage.get_filepath(amazon_raw_id))

    amazon_directoy = os.path.dirname(file_storage.get_filepath(amazon_raw_id))
    print(amazon_directoy)
    
    originalno_to_rawid = amazon_reader.originalno_to_rawid
    interim_storage.write(originalno_to_rawid)
else:
    originalno_to_rawid = interim_storage.read()

print(len(originalno_to_rawid))

/tmp/InterimStorage/originalno_to_rawid.pickle.bz2
7911684


## Get original numbers for available embeddings

In [5]:
number_of_all_reviews = 7911684
first_available_rawid = 84090
print(number_of_all_reviews - first_available_rawid)

orig_numbers_with_emb = {} # faster lookup than list
for item in originalno_to_rawid.items():
    if(item[1] >= first_available_rawid):
        orig_numbers_with_emb[item[0]] = item[0]
print(len(orig_numbers_with_emb))

7827594
7827594


## Check available embeddings

In [6]:
i = 0
for item in AmazonReviewsReader(file_storage.get_filepath('amazon_gz_file'), 'fields', max_docs=-1, min_year=2000, max_year=2010):
    #score = int(float(item['score']))
    #year = datetime.fromtimestamp(int(item['time'])).year
    number = item['number']
    if not number in orig_numbers_with_emb.keys():
        print(number)
    elif i % 500000 == 0:
        print('[',i,']')
    i += 1

print(i)

#[ 0 ]
#[ 500000 ]
#[ 1000000 ]
#[ 1500000 ]
#[ 2000000 ]
#[ 2500000 ]
#[ 3000000 ]
#[ 3500000 ]
#[ 4000000 ]
#[ 4500000 ]
#[ 5000000 ]
#[ 5500000 ]
#[ 6000000 ]
# 6184084

[ 0 ]
[ 500000 ]
[ 1000000 ]
[ 1500000 ]
[ 2000000 ]
[ 2500000 ]
[ 3000000 ]
[ 3500000 ]
[ 4000000 ]
[ 4500000 ]
[ 5000000 ]
[ 5500000 ]
[ 6000000 ]
6184084


In [1]:
# test
for item in AmazonReviewsReader(file_storage.get_filepath('amazon_gz_file'), 'fields', max_docs=2, min_year=1999, max_year=1999):
    number = item['number']
    if not number in orig_numbers_with_emb:
        print(number)

NameError: name 'AmazonReviewsReader' is not defined

In [9]:
print(originalno_to_rawid[6184084])
# 6788845

6788845


In [7]:
print(amazon_reader.get_text(6788845, metadata=True))
print(amazon_reader.get_bow50(6788845, metadata=True))
print(amazon_reader.get_bow50(6788845, metadata=False))

['1/1', 4, datetime.datetime(2011, 9, 7, 2, 0), 6184084, 6788845]
['1/1', 4, datetime.datetime(2011, 9, 7, 2, 0), 6184084, 6788845]
[ 0.4414188   0.13644266  0.17432572  0.05185378 -0.05497933  0.59349114
 -0.34965947 -0.6253097   0.04176392  0.2635524  -0.5170942   0.07202169
  0.2396945   0.52059996 -0.09735541  0.61579585 -0.2858135   0.23667523
 -0.1768746   0.12101202 -0.05129867  0.16942544  0.1835532  -0.4737814
 -0.40994313  0.05580128  0.2903279   0.06606621 -0.21997757 -0.1915003
  0.04107895 -0.2892729  -0.23737499  0.17370878 -0.6612374  -0.5322156
  0.18494414 -0.21456932 -0.33918732  0.68996286 -0.15533851 -0.10204262
 -1.4976032   0.6711743   0.25197563  0.0146621  -0.32608336 -0.5272231
  0.07257106  0.42317134]


# -> test not working now. Maybe create dataset first, check available embeddings later